In [1]:
import numpy as np
import scipy as sp
from scipy import optimize
from scipy import stats
import matplotlib.pyplot as plt
import sys, copy, emcee, corner
%matplotlib inline

In [2]:
sys.path.append('./ligbind/')
import ligbind as lb

In [3]:
def wym_obj(guess,model,data,ligs,rtot):
    datac = np.concatenate(data)
    return (model(guess,ligs,rtot)-datac)

def fit_wym(guess,model,data,ligs,rtot,bnds):
        return sp.optimize.least_squares(wym_obj,guess,args=(model,data,ligs,rtot),bounds=bnds,method='trf',ftol=1e-14,gtol=1e-14,xtol=1e-14)
    
def wym_obj_wt(guess,model,data,ligs,rtot):
    datac = np.concatenate(data)
    return (model(guess,ligs,rtot)-datac)/datac

def fit_wym_wt(guess,model,data,ligs,rtot,bnds):
        return sp.optimize.least_squares(wym_obj_wt,guess,args=(model,data,ligs,rtot),bounds=bnds,method='trf',ftol=1e-14,gtol=1e-14,xtol=1e-14)

In [4]:
def wymfunc_fix(parm,lig,rtot):
    '''
    Model function for dimerizing, single-site receptors as derived in
    (Wyman and Gill, Binding and Linkage, 1990) and used by
    (Macdonald and Pike, ...EGF-binding...negative cooperativity...aggregating system, 2008).
    Takes numpy array of parameters [k11,k21,k22,l20], numpy array of free ligand concentrations,
    and total receptor concentration. Returns the fraction of receptor bound to ligand.

    To generate a data set, iterate over an array of desired receptor concentrations, e.g.:
    [wymfunc(parm,lig,i) for i in array_of_rtots]

    For use in curve fitting an objective function for use with scipy.optimize.least_squares
    is readily constructed as: (wymfun(parm_guess,lig,rtot) - actual_data)
    '''

    #ensure dimension/broadcasting compatibility of inputs
    if ((rtot.ndim > 0) and (rtot.ndim != lig.ndim)):
        rtot = rtot[:,None] #adds dimension so that it can be broadcast
    else:
        rtot = rtot

    #unpack parameters
    k11 = parm[0]
    k21 = parm[1]
    k22 = k21/4.
    l20 = parm[2]

    ### START MODEL FUNCTION ###

    #calculate concentration of free (unoccupied) receptor
    rfree = (((-1 - k11*lig)) + \
    ((np.square((1 + k11*lig)) + \
    8.*l20*rtot*(1 + k21*lig + k21*k22*(np.square(lig)))))**0.5) \
    / (4*l20*(1 + k21*lig + k21*k22*(np.square(lig))))

    #calculate bound fraction
    bfrac = (k11*lig + l20*k21*rfree*lig + \
    2*l20*k21*k22*rfree*(np.square(lig))) \
    / (1 + 2*l20*rfree + k11*lig + \
    2*l20*k21*rfree*lig + 2*l20*k21*k22*rfree*(np.square(lig)))

    ### END MODEL FUNCTION ###

    #flatten output
    if ((rtot.ndim > 0) and (rtot.ndim != lig.ndim)):
        return bfrac.flatten()
    elif (rtot.ndim == 0):
        return bfrac.flatten()
    else:
        return np.concatenate(bfrac)

In [27]:
from itertools import product

def parmsetter(k1):
    base = np.array([k1,])
    k2s = np.array([0.1,1.,10.])*base
    k21s = np.array([1000.,100.,10.,0.1,0.01,0.001])*base
    ls = np.array([10.,100.,1000.])*base
    
    things = [base,k2s,k21s,ls]
    
    b = list(product(*things))
    
    c = np.array(b)
    
    c[:,2]=c[:,1]/4.
    
    neuts = np.array(list(set([tuple(i.tolist()) for i in c])))
    
    all_sets = np.concatenate([c,neuts])
    
    return all_sets

In [31]:
parmsets = parmsetter(5)

In [36]:
sim_n05 = [lb.WymSim(sets=1000,reps=1,parms=i,noise=0.05) for i in parmsets]

In [40]:
guess = np.array([5.,2.,0.5,500.])
guess_fix = np.array([5.,2.,500.])

In [41]:
lsbnds = (tuple(1e-5 for i in range(4)),tuple(1e4 for i in range(4)))
lsbnds_fix = (tuple(1e-5 for i in range(3)),tuple(1e4 for i in range(3)))

In [42]:
res05_full_wt = [np.array([fit_wym_wt(guess,lb.models.wymfunc,i,j.ligs,j.rtot,lsbnds) for i in j.meanset]) for j in sim_n05]

In [51]:
res05_fix_wt = [np.array([fit_wym_wt(guess_fix,wymfunc_fix,i,j.ligs,j.rtot,lsbnds_fix) for i in j.meanset]) for j in sim_n05]